# Pytorch Basic 2：自动求梯度（Autograd）

PyTorch的``Autograd``模块实现了深度学习的算法中的反向传播求导数，在``张量（Tensor类）``上的所有操作，Autograd都能为他们自动提供微分，简化了手动计算导数的复杂过程。

在0.4以前的版本中，Pytorch使用Variable类来自动计算所有的梯度Variable类主要包含三个属性：

data：保存Variable所包含的Tensor；

``grad``：保存``data对应的梯度``，grad也是个Variable，而不是Tensor，它和data的形状一样；

``grad_fn``：指向一个Function对象，这个Function用来反向传播计算输入的梯度。

从0.4起， Variable 正式合并入Tensor类，通过Variable嵌套实现的自动微分功能已经整合进入了Tensor类中。

虽然为了代码的兼容性还是可以使用Variable(tensor)这种方式进行嵌套，但是这个操作其实什么都没做。

所以，以后的代码建议直接使用Tensor类进行操作，因为官方文档中已经将Variable设置成过期模块。

要想通过``Tensor``类本身就支持了使用``autograd``功能，只需要设置.``requires_grad=True``

Variable类中的的`grad`和`grad_fn`属性已经整合进入了Tensor类中

即，现在直接用Tensor就可以自动求梯度了

In [1]:
import torch
print(torch.__version__)

1.10.0+cu111


## Autograd
在张量创建时，通过设置``requires_grad=Ture``来告诉Pytorch需要对该张量进行自动求导，PyTorch会记录该张量的每一步操作历史并自动计算

In [2]:
x=torch.rand(5,5,requires_grad=True)
print(x)

y=torch.rand(5,5,requires_grad=True)
print(y)

tensor([[0.8984, 0.1947, 0.9008, 0.3528, 0.4726],
        [0.0042, 0.3750, 0.1714, 0.6831, 0.3804],
        [0.3625, 0.2809, 0.1034, 0.8454, 0.8264],
        [0.5135, 0.1154, 0.9782, 0.2369, 0.4671],
        [0.4736, 0.2118, 0.4727, 0.4840, 0.4009]], requires_grad=True)
tensor([[0.0084, 0.7092, 0.7782, 0.3673, 0.2350],
        [0.9813, 0.9417, 0.0787, 0.4513, 0.1523],
        [0.9955, 0.8125, 0.6706, 0.0885, 0.1878],
        [0.5417, 0.2326, 0.1560, 0.7495, 0.6755],
        [0.7615, 0.0289, 0.7057, 0.2156, 0.7761]], requires_grad=True)


PyTorch会自动追踪和记录对与张量的所有操作，当计算完成后调用`.backward()`方法自动计算梯度并且将计算结果保存到`grad属性`中。

In [3]:
z=torch.sum(x+y) ## torch.sum() 对一个tensor中所有元素求和
print(z) ## 注意此时的z是一个标量

tensor(23.5075, grad_fn=<SumBackward0>)


在张量进行操作后，`grad_fn`已经被赋予了一个新的函数，这个函数引用了一个创建了这个`Tensor类`的`Function对象`。
Tensor和Function互相连接生成了一个非循环图(计算节点)，它记录并且编码了完整的计算历史。每个`张量`都有一个`.grad_fn属性`，如果这个`张量是用户手动创建`的那么这个张量的`grad_fn是None`。

下面我们来调用反向传播函数，计算其梯度

## 简单的自动求导

In [4]:
z.backward()
print(x.grad,'\t',y.grad) ## 由于z是一个标量，且x,y是叶子节点，所以可以完成自动求导

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]) 	 tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])


如果`Tensor`类表示的是一个`标量`（即它`包含一个元素的张量`），则`不需要`为backward()指定任何参数，但是如果它`有更多的元素`，则`需要指定一个gradient参数`，它是`形状匹配的张量`。

以上的 `z.backward()`相当于是`z.backward(torch.tensor(1.))`的简写。
这种参数常出现在图像分类中的单标签分类，输出一个标量代表图像的标签。

ps：在实际的网络中，我们一般是定义一个loss，然后正向传播得到这个loss，loss就是一个标量，那时候直接`loss.backward()`就可以了

## 复杂的自动求导

In [5]:
x = torch.rand(5, 5, requires_grad=True)
y = torch.rand(5, 5, requires_grad=True)
z= x**2+y**3
print(z.size())

torch.Size([5, 5])


我们的返回值(z)不是一个标量，所以需要输入一个`大小相同`的张量作为参数，这里我们用`ones_like`函数根据x生成一个张量

In [6]:
z.backward(torch.ones_like(x)) ## 和x同型的全为1的tensor
print(x.grad) ## 求Jacobian矩阵
#print(y.grad) ## 会报错

tensor([[1.6700, 1.6516, 1.4439, 0.9153, 0.2578],
        [0.5267, 1.3706, 1.4177, 0.2592, 0.2190],
        [0.2655, 0.9147, 1.3577, 0.3641, 1.7151],
        [1.1804, 1.3216, 0.5222, 0.5130, 0.1340],
        [0.3530, 1.2496, 0.3577, 1.8426, 0.0355]])


我们可以使用`with torch.no_grad()`上下文管理器`临时禁止`对已设置`requires_grad=True`的张量进行`自动求导`。这个方法在`测试集计算准确率`的时候会经常用到。

例如：

In [7]:
with torch.no_grad():
    print((x+y*2).requires_grad)

## 在测试集计算准确率的时候，一样进行正向传播，如果没有with torch.no_grad()，pytorch还是会去计算那些被设置为需要计算梯度的tensor

False


使用`.no_grad()`进行嵌套后，代码不会跟踪历史记录，也就是说保存的这部分记录会减少内存的使用量并且会加快少许的运算速度。

## Autograd 过程解析
为了说明Pytorch的自动求导原理，我们来尝试分析一下PyTorch的源代码，虽然Pytorch的 Tensor和 TensorBase都是使用CPP来实现的，但是可以使用一些Python的一些方法查看这些对象在Python的属性和状态。
 Python的 `dir()` 返回参数的属性、方法列表。`z`是一个Tensor变量，看看里面有哪些成员变量。

In [ ]:
dir(z)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '

返回很多，我们直接排除掉一些Python中特殊方法（以__开头和结束的）和私有方法（以_开头的，直接看几个比较主要的属性：

`.is_leaf`：记录是否是`叶子节点`。通过这个属性来确定这个变量的类型
在官方文档中所说的“graph leaves”，“leaf variables”，都是指像`x`，`y`这样的手动创建的、而非运算得到的变量，这些变量成为`创建变量`。

像`z`这样的，是通过计算后得到的结果称为`结果变量`。

一个变量是创建变量还是结果变量是通过`.is_leaf`来获取的。

In [ ]:
print("x.is_leaf="+str(x.is_leaf))
print("z.is_leaf="+str(z.is_leaf))

x.is_leaf=True
z.is_leaf=False


`x`是`手动创建`的没有通过计算，所以他被认为是一个`叶子节点`也就是一个创建变量，而`z`是通过`x`与`y`的一系列`计算得到`的，所以不是叶子结点也就是`结果变量`。

为什么我们执行`z.backward()`方法会更新`x.grad`和`y.grad`呢？
`.grad_fn`属性记录的就是这部分的操作，虽然`.backward()`方法也是CPP实现的，但是可以通过Python来进行简单的探索。

`grad_fn`：记录并且编码了完整的计算历史

In [ ]:
z.grad_fn

`grad_fn`是一个`AddBackward0`类型的变量 `AddBackward0`这个类也是用Cpp来写的，但是我们从名字里就能够大概知道，他是加法(ADD)的反向传播（Backward），看看里面有些什么东西

In [ ]:
dir(z.grad_fn)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_register_hook_dict',
 '_saved_alpha',
 'metadata',
 'name',
 'next_functions',
 'register_hook',
 'requires_grad']

`next_functions`就是`grad_fn`的精华

In [8]:
print(z.grad_fn)
print(z.grad_fn.next_functions)

((<PowBackward0 object at 0x7fccd0e5ded0>, 0), (<PowBackward0 object at 0x7fccd0e5d450>, 0))


`next_functions`是一个tuple of tuple of PowBackward0 and int。

为什么是2个tuple ？
因为我们的操作是`z= x**2+y**3` 刚才的`AddBackward0`是相加，而前面的操作是乘方 `PowBackward0`。tuple第一个元素就是x相关的操作记录。

即，第一个PowBackward0是x的操作记录，第二个是y的操作记录，`z=x**2+y**3`，所以`z.grad_fn.next_functions`就是记录了`z.grad_fn`的上一个操作记录（或者对反向传播来说是下一个），基于这种记录函数的办法，才能完成自动求导。

In [11]:
x1=torch.ones_like(x)
z=x**2+y**3+x1
z.backward(torch.ones_like(x))
print(z.grad_fn)
print(x1.grad_fn)
print(z.grad_fn.next_functions)

None
((<AddBackward0 object at 0x7fccd0e10490>, 0), (None, 0))


可以看到，新加了一个`x1`，由于x1这个tensor是叶子节点，对于`z.grad_fn.next_functions`，就包括了`x1.grad_fn`，为`none`

In [13]:
xg = z.grad_fn.next_functions[0][0] ## z.grad_fn.next_functions[0][0]（z.grad_fn.next_functions的第1个元组的第一个元素，那就是x.grad_fn属性）
dir(xg)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_register_hook_dict',
 '_saved_alpha',
 'metadata',
 'name',
 'next_functions',
 'register_hook',
 'requires_grad']

In [21]:
x_leaf=xg.next_functions[0][0]
x_leaf=x_leaf.next_functions[0][0] ## 这里由于我中间加了一个实验，所以多了一层，所以要对z找两个 .next_function[0][0]才回到x.grad_fn
print(type(x_leaf))
print(x.is_leaf)

<class 'AccumulateGrad'>
True


在PyTorch的反向图计算中，`AccumulateGrad`类型代表的就是叶子节点类型，也就是计算图终止节点。`AccumulateGrad`类中有一个`.variable`属性指向叶子节点。

这个`AccumulatedGrad`类可以理解成是在`叶子节点`的tensor的`.next_function`属性，相当于一个链表的终止指针，指向了一个none

In [22]:
x_leaf.variable 

tensor([[0.8350, 0.8258, 0.7220, 0.4577, 0.1289],
        [0.2633, 0.6853, 0.7088, 0.1296, 0.1095],
        [0.1328, 0.4573, 0.6789, 0.1820, 0.8575],
        [0.5902, 0.6608, 0.2611, 0.2565, 0.0670],
        [0.1765, 0.6248, 0.1788, 0.9213, 0.0177]], requires_grad=True)

这个`.variable`的属性就是我们的生成的变量`x`

In [23]:
print("x_leaf.variable的id:"+str(id(x_leaf.variable)))
print("x的id:"+str(id(x)))

x_leaf.variable的id:140517655156400
x的id:140517655156400


assert(id(x_leaf.variable)==id(x))

这样整个规程就很清晰了：

1. 当我们执行`z.backward()`的时候。这个操作将调用z里面的`grad_fn`这个属性，执行求导的操作。
2. 这个操作将`遍历grad_fn`的`next_functions`，然后分别取出里面的Function（AccumulateGrad），执行求导操作。这部分是一个`递归`的过程直到最后类型为`叶子节点`。
3. 计算出结果以后，将结果保存到他们对应的variable这个变量所引用的对象（x和y）的 `grad`这个属性里面。
4. 求导结束。所有的叶节点的grad变量都得到了相应的更新

最终当我们执行完`z.backward()`之后，a和b里面的grad值就得到了更新。

简单一句话：遍历`z.next_functions`直到叶子节点，保存所有中间计算到的梯度到各自的`.grad`里面

## 扩展Autograd
如果需要`自定义autograd`扩展新的功能，就需要`扩展Function类`。因为Function使用autograd来计算结果和梯度，并对操作历史进行编码。
在Function类中最主要的方法就是`forward()`和`backward()`他们分别代表了`前向传播`和`反向传播`。

一个自定义的Function需要一下三个方法：

    __init__ (optional)：如果这个操作需要额外的参数则需要定义这个Function的构造函数，不需要的话可以忽略。
    
    forward()：执行前向传播的计算代码
    
    backward()：反向传播时梯度计算的代码。 参数的个数和forward返回值的个数一样，每个参数代表传回到此操作的梯度。
        



In [25]:
# 引入Function便于扩展
from torch.autograd.function import Function

In [27]:
# 定义一个乘以常数的操作(输入参数是张量)
# 方法必须是静态方法，所以要加上@staticmethod 
class MulConstant(Function):
    @staticmethod ## 声明是静态方法
    def forward(ctx, tensor, constant):
        # ctx 用来保存信息这里类似self，并且ctx的属性可以在backward中调用
        ctx.constant=constant
        return tensor *constant
    @staticmethod 
    def backward(ctx, grad_output):
        # 返回的参数要与输入的参数一样.
        # 第一个输入为3x3的张量，第二个为一个常数
        # 常数的梯度必须是 None.
        return grad_output, None 

定义完我们的新操作后，我们来进行测试

In [28]:
a=torch.rand(3,3,requires_grad=True)
b=MulConstant.apply(a,5)
print("a:"+str(a))
print("b:"+str(b)) # b为a的元素乘以5

a:tensor([[0.7246, 0.6352, 0.3572],
        [0.4215, 0.1447, 0.5872],
        [0.7652, 0.3028, 0.3580]], requires_grad=True)
b:tensor([[3.6232, 3.1758, 1.7859],
        [2.1073, 0.7236, 2.9358],
        [3.8259, 1.5140, 1.7899]], grad_fn=<MulConstantBackward>)


反向传播，返回值不是标量，所以`backward`方法需要参数

In [29]:
b.backward(torch.ones_like(a))

In [30]:
a.grad

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

梯度为1。



## 总结
①autograd是基于tensor的`.next_function`属性来记录操作，`.grad来保存结果`

②对于计算结果如果不是一个标量，那么反向传播的时候需要指定一个和要求的梯度的叶子节点同型的tensor

③扩展Autog